In [139]:
import pandas as pd
import os
import numpy as np

SOURCE_DIR = 'G:\\Mugshots'
DEST_DIR = '../../data'
FILENAME1 = 'photoFeltGrønngylt_complete2020_addbrygga.csv'
FILENAME2 = 'RecapPIerFlødevigen.csv'

dateparser = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')

df1 = pd.read_csv(os.path.join(SOURCE_DIR, FILENAME1),
                 encoding='utf-8',
                 sep=';',
                 parse_dates=['date'],
                 date_parser=dateparser)

df2 = pd.read_csv(os.path.join(SOURCE_DIR, FILENAME2),
                 encoding='latin-1',
                 sep=';',
                 parse_dates=['date'],
                 date_parser=dateparser)

df = pd.concat([df1, df2], ignore_index=True)

df.dtypes

<ipython-input-139-8e3b57501b8e>:10: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  dateparser = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')


pit                    object
løpernummer            object
species                object
length                float64
sex                    object
spawning              float64
recap                 float64
prefix                 object
photostart              int64
photostop             float64
date           datetime64[ns]
Period                 object
dayseq                float64
area                   object
crypto                float64
skade                 float64
sexorig                object
dataset                object
nfotos                  int64
photoObs              float64
Temp_løp              float64
dtype: object

In [140]:
df.head()

,pit,løpernummer,species,length,sex,spawning,recap,prefix,photostart,photostop,...,Period,dayseq,area,crypto,skade,sexorig,dataset,nfotos,photoObs,Temp_løp
0,131073,25900,grønngylt,147.0,f,0.0,0.0,P702,4337,4338.0,...,10,1279.0,B,NaN,NaN,f,FieldMain,2,1.0,NaN
1,201164,25403,grønngylt,210.0,m,2.0,1.0,P701,3930,3931.0,...,10,1278.0,B,NaN,NaN,m,FieldMain,2,1.0,NaN
2,201179,10083,grønngylt,162.0,m,1.0,1.0,P707,1355,1356.0,...,3,553.0,L,NaN,NaN,m,FieldMain,2,1.0,NaN
3,201230,10432,grønngylt,136.0,f,0.0,1.0,P709,1573,1574.0,...,3,555.0,L,NaN,NaN,f,FieldMain,2,1.0,NaN
4,201262,1,grønngylt,183.0,m,0.0,NaN,PA280,542,543.0,...,Oct,NaN,NaN,NaN,NaN,NaN,LabMain,2,1.0,NaN


In [141]:
df['species'] = df['species'].apply(lambda x: 'grønngylt' if x == 'Grønngylt' else x)
df['species'].value_counts()

grønngylt    5181
Name: species, dtype: int64

In [142]:
# Oprette kolonne for dag og måned
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day
df.head()

,pit,løpernummer,species,length,sex,spawning,recap,prefix,photostart,photostop,...,area,crypto,skade,sexorig,dataset,nfotos,photoObs,Temp_løp,month,day
0,131073,25900,grønngylt,147.0,f,0.0,0.0,P702,4337,4338.0,...,B,NaN,NaN,f,FieldMain,2,1.0,NaN,7,2
1,201164,25403,grønngylt,210.0,m,2.0,1.0,P701,3930,3931.0,...,B,NaN,NaN,m,FieldMain,2,1.0,NaN,7,1
2,201179,10083,grønngylt,162.0,m,1.0,1.0,P707,1355,1356.0,...,L,NaN,NaN,m,FieldMain,2,1.0,NaN,7,7
3,201230,10432,grønngylt,136.0,f,0.0,1.0,P709,1573,1574.0,...,L,NaN,NaN,f,FieldMain,2,1.0,NaN,7,9
4,201262,1,grønngylt,183.0,m,0.0,NaN,PA280,542,543.0,...,NaN,NaN,NaN,NaN,LabMain,2,1.0,NaN,10,28


In [143]:
# Kolonner må være float for å kunne håndtere nan
print(df['photostart'].isna().sum())
print(df['photostop'].isna().sum())

0
184


In [144]:
def count_images(row):
    return 1 if pd.isna(row['photostop']) else row['photostop'] - row['photostart'] + 1

df['images_count'] = df.apply(lambda row: count_images(row), axis=1)
df['images_count'].value_counts()

2.0     4745
1.0      293
3.0      115
10.0      19
4.0        7
5.0        1
11.0       1
Name: images_count, dtype: int64

In [145]:
# Are nfotos and images_count the same?
df.loc[~(df['nfotos'] == df['images_count'])]

,pit,løpernummer,species,length,sex,spawning,recap,prefix,photostart,photostop,...,crypto,skade,sexorig,dataset,nfotos,photoObs,Temp_løp,month,day,images_count


In [146]:
# Finne ikke numeriske pit rader
df['pit'] = pd.to_numeric(df['pit'], errors='coerce')
df[df['pit'].apply(pd.isna)]

,pit,løpernummer,species,length,sex,spawning,recap,prefix,photostart,photostop,...,crypto,skade,sexorig,dataset,nfotos,photoObs,Temp_løp,month,day,images_count
4991,NaN,55,grønngylt,132.0,m,0.0,NaN,P501,1248,1249.0,...,NaN,NaN,NaN,LabMain,2,2.0,NaN,5,1,2.0
4992,NaN,43,grønngylt,138.0,m,0.0,NaN,P501,1224,1225.0,...,NaN,NaN,NaN,LabMain,2,2.0,NaN,5,1,2.0


In [147]:
df = df[df['pit'].apply(lambda x: not pd.isna(x))]
df.dtypes

pit                    float64
løpernummer             object
species                 object
length                 float64
sex                     object
spawning               float64
recap                  float64
prefix                  object
photostart               int64
photostop              float64
date            datetime64[ns]
Period                  object
dayseq                 float64
area                    object
crypto                 float64
skade                  float64
sexorig                 object
dataset                 object
nfotos                   int64
photoObs               float64
Temp_løp               float64
month                    int64
day                      int64
images_count           float64
dtype: object

In [148]:
# Hvor mange bilder det er på hvert individ
pit_group = df.groupby(['pit'])['nfotos']
pit_group.sum().value_counts()

2     3758
4      410
1      229
3      102
6       45
5       28
10      17
7        4
8        3
12       2
11       1
Name: nfotos, dtype: int64

In [149]:
df[df['Period'] == '2'].groupby(['prefix'])['prefix'].count()

prefix
P1160        2
P5110       13
P51100       8
_1160      197
_11700      28
_117000      4
Name: prefix, dtype: int64

In [160]:
# Hvor mange gange gjenfangster?
pit_count = df['pit'].value_counts()
pit_count = pit_count.to_dict()
df['pit_occurance'] = df.apply(lambda row: pit_count[row['pit']], axis=1)
df

,pit,løpernummer,species,length,sex,spawning,recap,prefix,photostart,photostop,...,skade,sexorig,dataset,nfotos,photoObs,Temp_løp,month,day,images_count,pit_occurance
0,131073.0,25900,grønngylt,147.0,f,0.0,0.0,P702,4337,4338.0,...,NaN,f,FieldMain,2,1.0,NaN,7,2,2.0,1
1,201164.0,25403,grønngylt,210.0,m,2.0,1.0,P701,3930,3931.0,...,NaN,m,FieldMain,2,1.0,NaN,7,1,2.0,1
2,201179.0,10083,grønngylt,162.0,m,1.0,1.0,P707,1355,1356.0,...,NaN,m,FieldMain,2,1.0,NaN,7,7,2.0,1
3,201230.0,10432,grønngylt,136.0,f,0.0,1.0,P709,1573,1574.0,...,NaN,f,FieldMain,2,1.0,NaN,7,9,2.0,1
4,201262.0,1,grønngylt,183.0,m,0.0,NaN,PA280,542,543.0,...,NaN,NaN,LabMain,2,1.0,NaN,10,28,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5176,434128.0,NaN,grønngylt,174.0,m,NaN,NaN,P82400,75,NaN,...,NaN,NaN,NaN,1,NaN,6.0,8,24,1.0,2
5177,434807.0,NaN,grønngylt,175.0,f,NaN,NaN,P8250,112,NaN,...,NaN,NaN,NaN,1,NaN,14.0,8,25,1.0,3
5178,434189.0,NaN,grønngylt,102.0,f,NaN,NaN,P8250,115,NaN,...,NaN,NaN,NaN,1,NaN,7.0,8,25,1.0,2
5179,434322.0,NaN,grønngylt,117.0,f,NaN,NaN,P8250,122,NaN,...,NaN,NaN,NaN,1,NaN,11.0,8,25,1.0,3


In [161]:
# Bare beholde de med flere gjenfangster
df[df['pit_occurance'] >= 2]

,pit,løpernummer,species,length,sex,spawning,recap,prefix,photostart,photostop,...,skade,sexorig,dataset,nfotos,photoObs,Temp_løp,month,day,images_count,pit_occurance
10,201506.0,194,grønngylt,137.0,f,0.0,NaN,PA290,934,935.0,...,NaN,NaN,LabMain,2,2.0,NaN,10,29,2.0,2
11,201506.0,127,grønngylt,138.0,f,0.0,NaN,P502,1391,1392.0,...,NaN,NaN,LabMain,2,2.0,NaN,5,2,2.0,2
21,202037.0,7355,grønngylt,120.0,m,0.0,1.0,_1160,801,802.0,...,NaN,m,FieldMain,2,2.0,NaN,5,16,2.0,2
22,202037.0,9425,grønngylt,131.0,m,0.0,1.0,P704,1637,1638.0,...,NaN,m,FieldMain,2,2.0,NaN,7,4,2.0,2
24,204295.0,9216,grønngylt,132.0,m,0.0,1.0,P703,1554,1555.0,...,NaN,m,FieldMain,2,2.0,NaN,7,3,2.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5176,434128.0,NaN,grønngylt,174.0,m,NaN,NaN,P82400,75,NaN,...,NaN,NaN,NaN,1,NaN,6.0,8,24,1.0,2
5177,434807.0,NaN,grønngylt,175.0,f,NaN,NaN,P8250,112,NaN,...,NaN,NaN,NaN,1,NaN,14.0,8,25,1.0,3
5178,434189.0,NaN,grønngylt,102.0,f,NaN,NaN,P8250,115,NaN,...,NaN,NaN,NaN,1,NaN,7.0,8,25,1.0,2
5179,434322.0,NaN,grønngylt,117.0,f,NaN,NaN,P8250,122,NaN,...,NaN,NaN,NaN,1,NaN,11.0,8,25,1.0,3
